#### Importación de paquetes

In [1]:
import cv2
import time
import FaceDetectors

In [2]:
# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

bigote = cv2.imread('Images/bigote.png', cv2.IMREAD_UNCHANGED)

gafas = cv2.imread('Images/gafas.png', cv2.IMREAD_UNCHANGED)
gafas = cv2.resize(gafas, (gafas.shape[1] // 4, gafas.shape[0] // 4))

pelo = cv2.imread('Images/pelo.png', cv2.IMREAD_UNCHANGED)
pelo = cv2.resize(pelo, (pelo.shape[1] // 2, pelo.shape[0] // 2)) 

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.DetectLargestFaceEyesDNN(frame)
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
            
                # Ajustar la posición del bigote
                x_bigote = shape[51][0] - int(bigote.shape[1] / 2)
                y_bigote = shape[51][1] - int(bigote.shape[0] / 2)

                x_gafas = shape[27][0] - int(gafas.shape[1] / 2)
                y_gafas = shape[27][1] - int(gafas.shape[0] / 2)

                # Ajustar la posición del pelo
                x_pelo = shape[27][0] - int(pelo.shape[1] / 2)
                y_pelo = shape[27][1] - int(pelo.shape[0] / 2) - 80

                # Asegurarse de que las coordenadas del bigote estén dentro de los límites de frame
                if x_bigote >= 0 and y_bigote >= 0 and (x_bigote + bigote.shape[1] <= frame.shape[1]) and (y_bigote + bigote.shape[0] <= frame.shape[0]):
                    bigote_resized = cv2.resize(bigote, (bigote.shape[1], bigote.shape[0]))  # Ajustar las dimensiones
                    alpha_bigote = bigote_resized[:, :, 3] / 255.0

                    for c in range(0, 3):
                        frame[y_bigote:y_bigote + bigote_resized.shape[0], x_bigote:x_bigote + bigote_resized.shape[1], c] = (
                            frame[y_bigote:y_bigote + bigote_resized.shape[0], x_bigote:x_bigote + bigote_resized.shape[1], c] * (1 - alpha_bigote) +
                            bigote_resized[:, :, c] * alpha_bigote
                        )

                # Asegurarse de que las coordenadas del pelo estén dentro de los límites de frame
                if x_gafas >= 0 and y_gafas >= 0 and (x_gafas + gafas.shape[1] <= frame.shape[1]) and (y_gafas + gafas.shape[0] <= frame.shape[0]):
                    gafas_resized = cv2.resize(gafas, (gafas.shape[1], gafas.shape[0]))  # Ajustar las dimensiones
                    alpha_gafas = gafas_resized[:, :, 3] / 255.0

                    for c in range(0, 3):
                        frame[y_gafas:y_gafas + gafas_resized.shape[0], x_gafas:x_gafas + gafas_resized.shape[1], c] = (
                            frame[y_gafas:y_gafas + gafas_resized.shape[0], x_gafas:x_gafas + gafas_resized.shape[1], c] * (1 - alpha_gafas) +
                            gafas_resized[:, :, c] * alpha_gafas
                        )

                # Asegurarse de que las coordenadas del pelo estén dentro de los límites de frame
                if x_pelo >= 0 and y_pelo >= 0 and (x_pelo + pelo.shape[1] <= frame.shape[1]) and (y_pelo + pelo.shape[0] <= frame.shape[0]):
                    pelo_resized = cv2.resize(pelo, (pelo.shape[1], pelo.shape[0]))  # Ajustar las dimensiones
                    alpha_pelo = pelo_resized[:, :, 3] / 255.0

                    for c in range(0, 3):
                        frame[y_pelo:y_pelo + pelo_resized.shape[0], x_pelo:x_pelo + pelo_resized.shape[1], c] = (
                            frame[y_pelo:y_pelo + pelo_resized.shape[0], x_pelo:x_pelo + pelo_resized.shape[1], c] * (1 - alpha_pelo) +
                            pelo_resized[:, :, c] * alpha_pelo
                        )

    # Show resulting image
    # cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    # if imodoF == 1 or imodoF == 2:
    #     cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    # # Face detector change
    # elif tec & 0xFF == ord('d'):
    #     imodoF = imodoF + 1
    #     if imodoF >= len(FDet.FaceDetectors):
    #         imodoF = 0
    # #Eye detector change
    # elif tec & 0xFF == ord('e'):
    #     imodoE = imodoE + 1
    #     if imodoE >= len(FDet.EyeDetectors):
    #         imodoE = 0

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
